# U-Net: Convolutional Networks for Biomedical Image Segmentation 
### Olaf Ronneberger, Philipp Fischer, and Thomas Brox

### Some general notes
* The original paper comes from back May 2015 that was the first release, so it's quite "old".
* At the time it was simply the best.
* It still is state-of-the-art at the time from some biomedical applications.

## 1.1 Introduction
* Discussing previous state-of-the-art apporaches.
* And how CNN outperform previous methods in visual recognition task.
* Highlighting the need for per-pixel-classification for some visual tasks (especially for biomedical images).
* Small number of training examples for biomedical task becuase of very specific field of expertise requiered.

In the later part of the introduction of this research paper, the authors discussed theory behind U-net architecture:

* More elegant fully convolutional network proposal.
* Base idea is to use downsampling path for **features extraction** followed by upsampling path for **precise localization** of these features in higher resolution layers.
* By concatenating downsampling feature maps with corresponding upsampling layers we help successive conv layers to assemble a more precise output.

(don't worry this is just the theory behind, when you see the implementation or the architecture it will make much more sense.)

One thing for mention here before we jump into the actual architecture is the *overlap tile strategy*:
* Input size > output size.
* Output segmentation map only contains the pixels for which the full context is available in the output image.
* Missing context on the edge/borders of input image is extrapolated by mirroring
![](res/overlap-tile-strategy.png)

For the test that this network was designed for, there were always very little training data available, so they had to come up with very excessive data augmentations strategy, another challenge that they were facing was working on more accurate separation of touching objects, just because when you have a lot of cells, many og them were touching each other, and it's easy for segmentation network to basically merge those cells, so they had to come up with a solution to penalize the network for doing so and to focus more on drawing those separation borders between cells. My intuition is that this idea comes after making a good error analysis, so maybe for our purpose we need to omit that part of the loss function, its looks quite specific-domain.

## 1.2 Network architecture